In [1]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Convolution2D, Reshape
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization

from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

print(tf.__version__)

2.0.0-dev20190604


In [2]:
tf.test.is_gpu_available()

True

In [3]:
import handshape_datasets as hd

DATASET_NAME = "lsa16"

data = hd.load(DATASET_NAME)

In [4]:
from sklearn.model_selection import train_test_split

features = data[0]
labels = data[1]['y']

x_train, x_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state=42)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
labels.shape

(800,)

In [6]:
'''
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0)
'''
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
datagen.fit(x_train)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
test_datagen.fit(x_train)

In [ ]:
from densenet import densenet_model
growth_rate=128
nb_layers=[6,12]

model = densenet_model(growth_rate=growth_rate, nb_layers=nb_layers)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(images, tf.float32), training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [ ]:
@tf.function
def test_step(images, labels):
    predictions = model(tf.cast(images, tf.float32), training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [ ]:
EPOCHS = 200
models_directory = 'trained-models'
results_directory = 'results'
results = 'epoch,loss,accuracy,test_loss,test_accuracy\n'
from datetime import datetime
now = datetime.now()

log_freq = 10
save_freq = 40
identifier = "tiny-{}-growth-{}-densenet-vanilla-data-".format('-'.join([str(i) for i in nb_layers] ), growth_rate)
             + now.strftime("%Y_%m_%d-%H:%M:%S")
train_summary_writer = tf.summary.create_file_writer('summaries/train/'+ identifier)
test_summary_writer = tf.summary.create_file_writer('summaries/test/'+ identifier)

for epoch in range(EPOCHS):

    batches = 0
    for images, labels in datagen.flow(x_train, y_train, batch_size=32):
        train_step(images, labels)
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

    batches = 0
    for test_images, test_labels in test_datagen.flow(x_test, y_test, batch_size=32, shuffle=False):
        test_step(test_images, test_labels)
        batches += 1
        if batches >= len(x_test) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

    if (epoch % log_freq == 0):
        results += '{},{},{},{},{}\n'.format(epoch,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100)
        
        print ('Epoch: {}, Train Loss: {}, Train Acc:{}, Test Loss: {}, Test Acc: {}'.format(epoch,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100))

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
            train_loss.reset_states()           
            train_accuracy.reset_states()           

        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            test_loss.reset_states()           
            test_accuracy.reset_states()           
        
        
    
    if (epoch % save_freq == 0):
        if not os.path.exists(models_directory):
            os.makedirs(models_directory)
        # serialize weights to HDF5
        model.save_weights(models_directory+"/{}_epoch{}.h5".format(identifier,epoch))
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
file = open(results_directory+'/results-'+ identifier + '.csv','w') 
file.write(results) 
file.close()